In [1]:
from datetime import datetime, timedelta
from glob import glob
from pathlib import Path

import pandas as pd

import os

INTERVAL = 60

用 src feature 產生時間序列資料

In [2]:
for filename in glob(Path('pch/20230329-20230331/src_feature/*').__str__()):
    print("Processing", filename, "...", " " * 20, end="\r")
    targetFile = filename.replace('src_feature', f'interval_{INTERVAL}_src_feature')
    if os.path.exists(targetFile):
        continue
    df = pd.read_parquet(filename)
    df[["timeStart"]] = df[["timeStart"]].astype(dtype='datetime64[ns]')
    start_time = datetime.fromisoformat("2023-03-29T00:00:00")
    end_time = start_time + timedelta(hours=72)
    result = []
    while start_time <= end_time:
        filtered = df[(df["timeStart"] >= start_time) & (df["timeStart"] < start_time + timedelta(seconds=INTERVAL))]
        result.append(
            {
                "timeStart": start_time,
                "srcIP": df["srcIP"].unique().tolist()[0],
                "packets": filtered["packets"].sum(),
                "bytes": filtered["bytes"].sum(),
                "flows": filtered["flows"].sum(),
                "bytes/packets": filtered["bytes"].sum() / filtered["packets"].sum() if filtered["packets"].sum() != 0 else 0,
                "flows/(bytes/packets)": filtered["flows"].sum() / (filtered["bytes"].sum() / filtered["packets"].sum()) if filtered["bytes"].sum() != 0 and filtered["packets"].sum() != 0 else 0,
                "nDstIP": filtered["nDstIP"].sum(),
                "nSrcPort": filtered["nSrcPort"].sum(),
                "nDstPort": filtered["nDstPort"].sum(),
            }
        )
        start_time += timedelta(seconds=INTERVAL)
    result = pd.DataFrame(result)
    # print(result)
    # break
    os.makedirs(os.path.dirname(targetFile), exist_ok=True)
    result.to_parquet(targetFile, index=False)

KeyboardInterrupt: 

用 dst feature 產生時間序列資料

In [ ]:
for filename in glob(Path('pch/20230329-20230331/dst_feature/*').__str__()):
    print("Processing", filename, "...", " " * 20, end="\r")
    targetFile = filename.replace('dst_feature', f'interval_{INTERVAL}_dst_feature')
    if os.path.exists(targetFile):
        continue
    df = pd.read_parquet(filename)
    df[["timeStart"]] = df[["timeStart"]].astype(dtype='datetime64[ns]')
    start_time = datetime.fromisoformat("2023-03-29T00:00:00")
    end_time = start_time + timedelta(hours=72)
    result = []
    while start_time <= end_time:
        filtered = df[(df["timeStart"] >= start_time) & (df["timeStart"] < start_time + timedelta(seconds=INTERVAL))]
        result.append(
            {
                "timeStart": start_time,
                "dstIP": df["dstIP"].unique().tolist()[0],
                "packets": filtered["packets"].sum(),
                "bytes": filtered["bytes"].sum(),
                "flows": filtered["flows"].sum(),
                "bytes/packets": filtered["bytes"].sum() / filtered["packets"].sum() if filtered["packets"].sum() != 0 else 0,
                "flows/(bytes/packets)": filtered["flows"].sum() / (filtered["bytes"].sum() / filtered["packets"].sum()) if filtered["bytes"].sum() != 0 and filtered["packets"].sum() != 0 else 0,
                "nSrcIP": filtered["nSrcIP"].sum(),
                "nSrcPort": filtered["nSrcPort"].sum(),
                "nDstPort": filtered["nDstPort"].sum(),
            }
        )
        start_time += timedelta(seconds=INTERVAL)
    result = pd.DataFrame(result)
    # print(result)
    # break
    os.makedirs(os.path.dirname(targetFile), exist_ok=True)
    result.to_parquet(targetFile, index=False)